In [1]:
import os, sys
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
sys.path.append("../..") # Adds higher directory to python modules path.

import tensorflow as tf
from tensorflow import keras
from keras_flops import get_flops 
import numpy as np

import brevis
from brevis import raw_models
from utils import Brevis_loss_final, growth_update, branch_conv2d, save_outputs
from utils import getPredictions_Energy, infer_result, infer_result_OOD, get_branched_flops
# os.environ['TF_DETERMINISTIC_OPS'] = '1'

/home/dunght/anaconda3/envs/tf2.2/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
../../brevis/core_v2.py:25: UserWarning: Logging module Neptune was not found, Cloud Logging is not enabled, check https://docs.neptune.ai/getting-started/installation for more information on how to set this up
  warnings.warn("Logging module Neptune was not found, Cloud Logging is not enabled, check https://docs.neptune.ai/getting-started/installation for more information on how to set this up")
2022-12-01 08:22:16.199102: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-12-01 08:22:16.227881: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 3492935000 Hz
2022-12-01 08:22:16.229882: I tensorflow/co

In [2]:
BASEMODEL_PATH = 'alexnet_cifar10_base.hdf5'
BRANCHY_PATH = "alexnet_cifar10_branchy.hdf5"
BREVIS_PATH = "alexnet_cifar10_brevis.hdf5"

## Model Determinism
If using Deterministic Operations, enable the following block to ensure that at each step the model begins with the same random seed.
This is particularlly helpful to directly compare hyperparameter changes for small improvements without the need to repeat multiple tests to get average results.
It is slightly slower but still fairly fast.

In [3]:
_DETERMINISTIC = False
# _DETERMINISTIC = True

_SEED = 42
if _DETERMINISTIC: 
    import random
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    _SEED = 42

def reset_seeds(seed = _SEED):
    random.seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)

## Load dataset
Datasets are loaded in batches of 32, input sizes of (32,32) and shuffled between epochs.
For Cifar datasets, The standard train and set sizes are used, with a separate 5k training images separated for validation set purposes.

In [4]:
INPUT_SIZE = 224
N_CHANNELS = 3
N_CLASSES = 10
DATASET = tf.keras.datasets.cifar10.load_data()
train_ds, test_ds, validation_ds = brevis.dataset.prepare.dataset(tf.keras.datasets.cifar10.load_data(),
                                                                32,5000,shuffle_size=15000,input_size=(INPUT_SIZE,INPUT_SIZE),
                                                                include_targets=False,num_outputs = N_CLASSES,reshuffle=True)
# INPUT_SIZE = 224
# N_CHANNELS = 3
# N_CLASSES = 5
# DATASET = 'cassava'

# train_ds, test_ds, validation_ds = brevis.dataset.prepare.dataset(DATASET,
#                                                             32,1000,shuffle_size=1500,input_size=(INPUT_SIZE,INPUT_SIZE),
#                                                             include_targets=False,num_outputs = N_CLASSES,reshuffle=True)


here


2022-12-01 08:22:17.378117: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:09:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.62GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2022-12-01 08:22:17.378216: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-12-01 08:22:17.378231: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-12-01 08:22:17.378243: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2022-12-01 08:22:17.378254: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10
2022-12-01 08:22:17.378265: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusolve

augment Dataset
targetsis : False
trainSize 45000
testSize 10000


In [26]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

In [30]:
train_images.shape, train_labels.shape

((50000, 32, 32, 3), (50000, 1))

In [5]:
_, test_ds100, _ = brevis.dataset.prepare.dataset(tf.keras.datasets.cifar100.load_data(label_mode="fine"),
                                                                32,5000,shuffle_size=15000,input_size=(INPUT_SIZE,INPUT_SIZE),
                                                                include_targets=False,num_outputs = 100,reshuffle=True)


here
augment Dataset
targetsis : False
trainSize 45000
testSize 10000


## From Scratch Model

In [6]:
base_model = raw_models.alexnet.get_model(INPUT_SIZE, N_CLASSES, N_CHANNELS)

In [7]:
flops = get_branched_flops(base_model)
print(f"FLOPS: {flops:.03} G")


2022-12-01 08:22:31.707786: I tensorflow/core/grappler/devices.cc:55] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2022-12-01 08:22:31.707928: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2022-12-01 08:22:31.709210: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:09:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.62GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2022-12-01 08:22:31.709258: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-12-01 08:22:31.709269: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-12-01 08:22:31.709278: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2022-12-01 08:22:31.709287: I te

Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
Parsing Inputs...

=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
scope: The nodes in the model graph are organized by their names, which is hierarchical like filesys

In [8]:
base_model.compile(optimizer='adam', 
                loss='categorical_crossentropy',
                metrics = ['accuracy'])

base_model.summary()
earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=5, verbose=1, mode='auto')

base_model.fit(train_ds, epochs=1, validation_data = validation_ds, batch_size=64,callbacks=[earlystop])

loss, accuracy = base_model.evaluate(test_ds, batch_size=64)
base_model.save(BASEMODEL_PATH)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 54, 54, 96)        34944     
_________________________________________________________________
batch_normalization (BatchNo (None, 54, 54, 96)        384       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 26, 26, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 26, 26, 256)       614656    
_________________________________________________________________
batch_normalization_1 (Batch (None, 26, 26, 256)       1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 384)       8

2022-12-01 08:22:35.344760: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-12-01 08:22:43.651799: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7


313/313 [==============================] - 5s 17ms/step - loss: 1.7249 - accuracy: 0.4043


## Branchy Model

In [9]:
tf.keras.backend.clear_session()
if _DETERMINISTIC:
    reset_seeds()
    
### setup hyperparameter growth controls.

### Loss functions 
trunk_loss = keras.losses.CategoricalCrossentropy(from_logits=False)
CE_loss = keras.losses.CategoricalCrossentropy(from_logits=True)

### Build the Model
base_model = raw_models.alexnet.get_model(INPUT_SIZE, N_CLASSES, N_CHANNELS)
branchy_model = brevis.BranchModel(model=base_model, custom_objects={})
branchy_model = brevis.branches.branch.add_branches(branchy_model, 
                                            [lambda *args, **kwargs: branch_conv2d(*args, depths = [64, 256], **kwargs),
                                            lambda *args, **kwargs: branch_conv2d(*args, depths = [128, 512], **kwargs)],
                                           branchPoints =  ["max_pooling2d","max_pooling2d_1"],
                                            target_input=False,loop=False,num_outputs=N_CLASSES)
              
### Enable the training of the branches
branchy_model.setFrozenTraining(False)

### Compile the model. Provide either a single loss or an individual loss for each exit.
branchy_model.compile(loss=[trunk_loss,CE_loss,CE_loss], 
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.001,  beta_1=0.99,  beta_2=0.999,),
                metrics=['accuracy'])

# brevis_model = brevis.branches.branch.add_branches(
#                         brevis_model,
#                         [branch_conv2d , branch_conv2d],
#                         ["mixed0","mixed1"],
#                         target_input=False,loop=False,num_outputs=N_CLASSES)

# brevis_model.compile(loss=[trunk_loss,branch_loss,branch_loss], 
#                   optimizer=tf.optimizers.SGD(learning_rate=0.01, momentum=0.9),
#                    metrics=['accuracy'])

Matching Branchpoint by id number
Matching Branchpoint by name
add Branch to branch point  max_pooling2d
add Branch to branch point  max_pooling2d_1
Tensor("conv2d_input:0", shape=(None, 224, 224, 3), dtype=float32)
[<tf.Tensor 'dense_2/Identity:0' shape=(None, 10) dtype=float32>, <tf.Tensor 'branch_exit/Identity:0' shape=(None, 10) dtype=float32>, <tf.Tensor 'branch_exit_1/Identity:0' shape=(None, 10) dtype=float32>]
Setting Main Layers  and branch layers training to true


In [10]:
branchy_model.summary()

Model: "branch_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
conv2d_input (InputLayer)       [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 54, 54, 96)   34944       conv2d_input[0][0]               
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 54, 54, 96)   384         conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 26, 26, 96)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

## Train the model and its newly added branches

In [11]:
earlystop1 = tf.keras.callbacks.EarlyStopping(monitor='val_branch_exit_accuracy', min_delta=0.0001, patience=5, verbose=1, mode='auto',restore_best_weights=True)
earlystop2 = tf.keras.callbacks.EarlyStopping(monitor='val_branch_exit_1_accuracy', min_delta=0.0001, patience=5, verbose=1, mode='auto',restore_best_weights=True)
earlystop3 = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=5, verbose=1, mode='auto')
branchy_model.fit(train_ds, epochs = 1, validation_data=validation_ds, callbacks=[earlystop1, earlystop2, earlystop3])

1407/1407 [==============================] - 52s 37ms/step - loss: 6.0090 - dense_2_loss: 2.4668 - branch_exit_loss: 1.7416 - branch_exit_1_loss: 1.8006 - dense_2_accuracy: 0.2756 - branch_exit_accuracy: 0.4047 - branch_exit_1_accuracy: 0.3508 - val_loss: 5.0419 - val_dense_2_loss: 1.8408 - val_branch_exit_loss: 1.5843 - val_branch_exit_1_loss: 1.6168 - val_dense_2_accuracy: 0.3618 - val_branch_exit_accuracy: 0.4292 - val_branch_exit_1_accuracy: 0.4118


In [12]:
branchy_model.evaluate(test_ds)


313/313 [==============================] - 6s 18ms/step - loss: 5.0766 - dense_2_loss: 1.8180 - branch_exit_loss: 1.6358 - branch_exit_1_loss: 1.6227 - dense_2_accuracy: 0.3698 - branch_exit_accuracy: 0.4275 - branch_exit_1_accuracy: 0.4060


[5.076560974121094,
 1.818029522895813,
 1.6358327865600586,
 1.62269926071167,
 0.36980000138282776,
 0.42750000953674316,
 0.4059999883174896]

## Evaluate BranchyNet Results
Next follow the below steps to evaluate the trained model to determine efficency improvement.

### 1. Cacluate Model Flops
This calculator uses the inbuilt Tensorflow operation measurement and estimates the flop usage for each classfication exit.
Early exits are included in the calculation of later exits to represent the cost of evaluating at the earlier branch and then the output being rejected.

In [13]:
flops = get_branched_flops(branchy_model, ["branch_exit", "branch_exit_1", "dense_2"])

2022-12-01 08:24:49.417937: I tensorflow/core/grappler/devices.cc:55] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2022-12-01 08:24:49.418051: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2022-12-01 08:24:49.419205: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:09:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.62GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2022-12-01 08:24:49.419256: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-12-01 08:24:49.419269: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-12-01 08:24:49.419279: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2022-12-01 08:24:49.419289: I te

Parsing Inputs...

=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
scope: The nodes in the model graph are organized by their names, which is hierarchical like filesystem.
flops: Number of float operations. Note: Please read the implementation for the math

2022-12-01 08:24:49.782663: I tensorflow/core/grappler/devices.cc:55] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2022-12-01 08:24:49.782763: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2022-12-01 08:24:49.783922: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:09:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.62GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2022-12-01 08:24:49.783971: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-12-01 08:24:49.783982: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-12-01 08:24:49.783991: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2022-12-01 08:24:49.784000: I te

Parsing Inputs...

=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
scope: The nodes in the model graph are organized by their names, which is hierarchical like filesystem.
flops: Number of float operations. Note: Please read the implementation for the math

2022-12-01 08:24:50.328226: I tensorflow/core/grappler/devices.cc:55] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2022-12-01 08:24:50.328329: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2022-12-01 08:24:50.329343: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:09:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.62GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2022-12-01 08:24:50.329412: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-12-01 08:24:50.329425: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-12-01 08:24:50.329437: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2022-12-01 08:24:50.329448: I te

Parsing Inputs...

=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
scope: The nodes in the model graph are organized by their names, which is hierarchical like filesystem.
flops: Number of float operations. Note: Please read the implementation for the math

In [14]:
flops

[0.237901642, 1.092310996, 1.50447336]

### 3. Get Test set predictions

In [15]:
output_branchy_ID= getPredictions_Energy(branchy_model, test_ds,stopping_point=None)
output_branchy_OOD = getPredictions_Energy(branchy_model, test_ds100,stopping_point=None)

Processing branch 1: 10000it [00:27, 362.84it/s]
Processing branch 2: 10000it [00:27, 364.57it/s]
Processing branch 3: 10000it [00:27, 367.54it/s]
Processing branch 1: 10000it [00:27, 361.34it/s]
Processing branch 2: 10000it [00:26, 375.40it/s]
Processing branch 3: 10000it [00:27, 362.48it/s]


### 4. Evaluate Test Results


In [16]:
for metric in ["entropy", "uncert", "calibration"]:
    infer_result_OOD(output_branchy_ID, output_branchy_OOD, [metric], threshold='gmean', flops=flops)
    print("="*100)

metric:  entropy threshold:  gmean
m entropy
metric entropy
entropy  lr_auc 0.3685694352355627 Best Threshold=3.3190667629241943, G-Mean=0.5921168292960475, TPR=0.6010790225325293, FPR=0.4167117360735533
m entropy
metric entropy
entropy  lr_auc 0.30445306570647873 Best Threshold=2.035510540008545, G-Mean=0.6417340333224094, TPR=0.6520524017467249, FPR=0.3684210526315789
m entropy
metric entropy
entropy  lr_auc 0.3265507082317427 Best Threshold=2.1055169105529785, G-Mean=0.6212086745044112, TPR=0.6452861952861952, FPR=0.4019704433497537
   Exit_Name  ID_Inputs  OOD_Inputs  Test_Accuracy Threshold  Accepted ID  Accepted OOD  Accepted_Correct  Accepted_ID_Ratio  Acceptance_Accuracy
0     exit_1      10000       10000         0.3698   3.31907         4672          3396              2157           0.579078             0.267353
1     exit_2       5328        6604         0.4275   2.03551         1355          1294               747           0.511514             0.281993
2  Main_exit       3

### 5. Save the model


In [17]:
branchy_model.save(BRANCHY_PATH)

## BrevisNet Model
Now that we've established a baseline using the branchynet model, the next step is to build and evaluate the BrevisNet version.
This model will use a different loss function, enable the growth hyperparameter, and use "energy" as the evaluation value.




### Build the model


In [18]:
tf.keras.backend.clear_session()
if _DETERMINISTIC:
    reset_seeds()
### setup hyperparameter growth controls.
    ## always start growth >= 1 to allow the model at least one epoch to stabilise with just the basic loss.
growth_callback = growth_update(annealing_rate = 60, start_t = 0.,max_t = 1, starting_epoch =1, branch_names=["branch_exit_accuracy","branch_exit_1_accuracy"])
earlyStop = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",patience=4,restore_best_weights=True)

### Loss functions 
branch_loss = Brevis_loss_final(growth_callback)  ### the brevisnet loss function.
trunk_loss = keras.losses.CategoricalCrossentropy(from_logits=False) ### the standard loss function for the main exit
# CE_loss = keras.losses.CategoricalCrossentropy(from_logits=True)

### Build the Model
brevis_model = brevis.BranchModel(name=BASEMODEL_PATH, custom_objects={})
brevis_model = brevis.branches.branch.add_branches(brevis_model, 
                                            [lambda *args, **kwargs: branch_conv2d(*args, depths = [128, 512], **kwargs),
                                            lambda *args, **kwargs: branch_conv2d(*args, depths = [128, 512], **kwargs)],
                                           branchPoints =  ["max_pooling2d","max_pooling2d_1"],
                                            target_input=False,loop=False,num_outputs=N_CLASSES)
              
### Enable the training of the branches
brevis_model.setFrozenTraining(True)

### Compile the model. Provide either a single loss or an individual loss for each exit.
brevis_model.compile(loss=[trunk_loss,branch_loss,branch_loss], 
                # optimizer=tf.keras.optimizers.Adam(learning_rate=0.001,  beta_1=0.99,  beta_2=0.999,),
                optimizer=tf.optimizers.SGD(learning_rate=0.01, momentum=0.9),
                metrics=['accuracy'])

Matching Branchpoint by id number
Matching Branchpoint by name
add Branch to branch point  max_pooling2d
add Branch to branch point  max_pooling2d_1
Tensor("conv2d_input:0", shape=(None, 224, 224, 3), dtype=float32)
[<tf.Tensor 'dense_2/Identity:0' shape=(None, 10) dtype=float32>, <tf.Tensor 'branch_exit/Identity:0' shape=(None, 10) dtype=float32>, <tf.Tensor 'branch_exit_1/Identity:0' shape=(None, 10) dtype=float32>]
Freezing Main Layers and setting branch layers training to true


## Train the model and its newly added branches

In [19]:
earlystop1 = tf.keras.callbacks.EarlyStopping(monitor='val_branch_exit_accuracy', min_delta=0.0001, patience=5, verbose=1, mode='auto',restore_best_weights=True)
earlystop2 = tf.keras.callbacks.EarlyStopping(monitor='val_branch_exit_1_accuracy', min_delta=0.0001, patience=5, verbose=1, mode='auto',restore_best_weights=True)
brevis_model.fit(train_ds, epochs = 1, validation_data=validation_ds, callbacks=[earlystop1, earlystop2])

1407/1407 [==============================] - 49s 35ms/step - loss: 3.3297 - dense_2_loss: 1.9574 - branch_exit_loss: 0.6610 - branch_exit_1_loss: 0.7114 - dense_2_accuracy: 0.3672 - branch_exit_accuracy: 0.4786 - branch_exit_1_accuracy: 0.4248 - val_loss: 2.9257 - val_dense_2_loss: 1.7437 - val_branch_exit_loss: 0.5748 - val_branch_exit_1_loss: 0.6071 - val_dense_2_accuracy: 0.3922 - val_branch_exit_accuracy: 0.5666 - val_branch_exit_1_accuracy: 0.5348


In [20]:
brevis_model.evaluate(test_ds)


313/313 [==============================] - 6s 19ms/step - loss: 2.9173 - dense_2_loss: 1.7249 - branch_exit_loss: 0.5787 - branch_exit_1_loss: 0.6137 - dense_2_accuracy: 0.4043 - branch_exit_accuracy: 0.5610 - branch_exit_1_accuracy: 0.5283


[2.91733455657959,
 1.7248915433883667,
 0.5787474513053894,
 0.6136962175369263,
 0.4043000042438507,
 0.5609999895095825,
 0.5282999873161316]

## Evaluate BranchyNet Results
Next follow the below steps to evaluate the trained model to determine efficency improvement.

### 1. Cacluate Model Flops
This calculator uses the inbuilt Tensorflow operation measurement and estimates the flop usage for each classfication exit.
Early exits are included in the calculation of later exits to represent the cost of evaluating at the earlier branch and then the output being rejected.

In [21]:
flops = get_branched_flops(brevis_model, ["branch_exit", "branch_exit_1", "dense_2"])

2022-12-01 08:29:03.386634: I tensorflow/core/grappler/devices.cc:55] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2022-12-01 08:29:03.386746: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2022-12-01 08:29:03.387604: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:09:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.62GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2022-12-01 08:29:03.387653: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-12-01 08:29:03.387664: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-12-01 08:29:03.387674: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2022-12-01 08:29:03.387683: I te

Parsing Inputs...

=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
scope: The nodes in the model graph are organized by their names, which is hierarchical like filesystem.
flops: Number of float operations. Note: Please read the implementation for the math

2022-12-01 08:29:04.364244: I tensorflow/core/grappler/devices.cc:55] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2022-12-01 08:29:04.364347: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2022-12-01 08:29:04.365519: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:09:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.62GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2022-12-01 08:29:04.365577: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-12-01 08:29:04.365590: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-12-01 08:29:04.365601: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2022-12-01 08:29:04.365611: I te

Parsing Inputs...

=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
scope: The nodes in the model graph are organized by their names, which is hierarchical like filesystem.
flops: Number of float operations. Note: Please read the implementation for the math

2022-12-01 08:29:05.579680: I tensorflow/core/grappler/devices.cc:55] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2022-12-01 08:29:05.579808: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2022-12-01 08:29:05.580653: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:09:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.62GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2022-12-01 08:29:05.580702: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-12-01 08:29:05.580713: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-12-01 08:29:05.580722: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2022-12-01 08:29:05.580731: I te

Parsing Inputs...

=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
scope: The nodes in the model graph are organized by their names, which is hierarchical like filesystem.
flops: Number of float operations. Note: Please read the implementation for the math

In [22]:
flops

[0.290597706, 1.14500706, 1.557169424]

### 3. Get Test set predictions

In [23]:
output_brevis_ID= getPredictions_Energy(brevis_model, test_ds,stopping_point=None)
output_brevis_OOD = getPredictions_Energy(brevis_model, test_ds100,stopping_point=None)

Processing branch 1: 10000it [00:25, 392.33it/s]
Processing branch 2: 10000it [00:26, 376.76it/s]
Processing branch 3: 10000it [00:27, 369.90it/s]
Processing branch 1: 10000it [00:27, 361.53it/s]
Processing branch 2: 10000it [00:27, 357.91it/s]
Processing branch 3: 10000it [00:27, 363.46it/s]


### 4. Evaluate Test Results


In [24]:
for metric in ["energy"]:
    infer_result_OOD(output_brevis_ID, output_brevis_OOD, [metric], threshold='gmean', flops=flops)
    print("="*100)

metric:  energy threshold:  gmean
m energy
metric energy
energy  lr_auc 0.31811623752068324 Best Threshold=-2.410576820373535, G-Mean=0.6383267107010017, TPR=0.673493369145543, FPR=0.3950037101162503
m energy
metric energy
energy  lr_auc 0.2752480520060582 Best Threshold=-54.452335357666016, G-Mean=0.6669990624131235, TPR=0.6835990888382688, FPR=0.34919786096256683
m energy
metric energy
energy  lr_auc 0.3061492675475446 Best Threshold=-58.750389099121094, G-Mean=0.6436317523526641, TPR=0.6809412762348951, FPR=0.39163354154836266
   Exit_Name  ID_Inputs  OOD_Inputs  Test_Accuracy Threshold  Accepted ID  Accepted OOD  Accepted_Correct  Accepted_ID_Ratio  Acceptance_Accuracy
0     exit_1      10000       10000         0.4043  -2.41058         4392          3309              2446           0.570316             0.317621
1     exit_2       5608        6691         0.5610  -54.4523         2136          1455              1448           0.594820             0.403230
2  Main_exit       3472   

### 5. Save the model

In [25]:
brevis_model.save(BREVIS_PATH)